In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
df=pd.read_csv('train.txt',sep=';' , header= None , names=['text','emotions'])

In [3]:
df.head()

,text,emotions
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [5]:
df.isnull().sum()

text        0
emotions    0
dtype: int64

In [8]:
unique_emotions=df['emotions'].unique()
emotion_numbers={}
i=0
for emo in unique_emotions:
    emotion_numbers[emo]=i
    i+=1

df['emotions']=df['emotions'].map(emotion_numbers)

In [9]:
df

,text,emotions
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,1
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,1
...,...,...
15995,i just had a very brief time in the beanbag an...,0
15996,i am now turning and i feel pathetic that i am...,0
15997,i feel strong and good overall,5
15998,i feel like this was such a rude comment and i...,1


In [10]:
df['text']=df['text'].apply(lambda x : x.lower())

In [12]:
import string 


In [13]:
def remove_punct(txt):
    return txt.translate(str.maketrans('','',string.punctuation))

In [14]:
df['text']= df['text'].apply(remove_punct)

In [ ]:
def remove_number(txt):
    new= ''
    for i in txt:
        if not  i.isdigit():
            new=new +i
    return new

df['text']=df['text'].apply(remove_number)

In [18]:
def remove_emogi(txt):
    new=''
    for i in txt:
        if i.isascii():
            new += i
    return new

df['text']=df['text'].apply(remove_emogi) 

In [32]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [26]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [33]:
stop_words=set(stopwords.words('english'))
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 "he's",
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 "i'll",
 "i'm",
 "i've",
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'on

In [22]:
df.loc[1]['text']

'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake'

In [36]:
def remove(txt):
    words = txt.split()
    cleaned = [i for i in words if i.lower() not in stop_words]  # also convert to lowercase
    return ' '.join(cleaned)



In [38]:
df['text'] = df['text'].apply(remove)


In [39]:
df.loc[1]['text']

'go feeling hopeless damned hopeful around someone cares awake'

In [40]:
from sklearn.model_selection import train_test_split


In [41]:
X_train , X_test , y_train , y_test = train_test_split(df['text'] ,df['emotions'], random_state=42 , test_size= 0.20)

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer , CountVectorizer

In [43]:
bow_vectorizer=CountVectorizer()

In [50]:
X_train_bow=bow_vectorizer.fit_transform(X_train)
X_test_bow=bow_vectorizer.transform(X_test)

In [51]:
from sklearn.naive_bayes import MultinomialNB 
from sklearn.metrics import accuracy_score


In [52]:
nb_model=MultinomialNB()

In [54]:
nb_model.fit(X_train_bow, y_train)

MultinomialNB()

In [55]:
pred_nb=nb_model.predict(X_test_bow)

In [56]:
accuracy_score(y_test,pred_nb)

0.768125

In [57]:
Tfidf_Vectorizer=TfidfVectorizer()

In [58]:
X_train_tfid=Tfidf_Vectorizer.fit_transform(X_train)
X_test_tfid=Tfidf_Vectorizer.transform(X_test)

In [59]:
nb2_model=MultinomialNB()
nb2_model.fit(X_train_tfid,y_train)

MultinomialNB()

In [61]:
y_pred_nb2=nb2_model.predict(X_test_tfid)

In [62]:
accuracy_score(y_test , y_pred_nb2)

0.6609375

In [63]:
from sklearn.linear_model import LogisticRegression

In [64]:
logistic_model=LogisticRegression(max_iter=1000)


In [65]:
logistic_model.fit(X_train_tfid,y_train)

LogisticRegression(max_iter=1000)

In [66]:
log_pred=logistic_model.predict(X_test_tfid)

In [67]:
accuracy_score(y_test,log_pred)

0.8628125